In [1]:
from google.colab import files
uploaded = files.upload()


Saving housing_processed.csv to housing_processed.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
df = pd.read_csv("housing_processed.csv")
X = df.drop(columns=["median_house_value"]).values
median_value = df["median_house_value"].median()
y = (df["median_house_value"] > median_value).astype(int).values
model = LogisticRegression(max_iter=5000)
model.fit(X, y)
w_sklearn = model.coef_[0]
b_sklearn = model.intercept_[0]

print("w (sklearn) =", w_sklearn)
print("b (sklearn) =", b_sklearn)


w (sklearn) = [-7.29471623 -6.38608021  1.05781331 -1.90222234  3.79093671 -5.15115236
  4.10622343  9.39266725 -1.82102173  0.77514795 -0.72605626 -0.5165456 ]
b (sklearn) = 1.5898353017178655


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
df = pd.read_csv("housing_processed.csv")

X = df.drop(columns=["median_house_value"]).values
median_value = df["median_house_value"].median()
y = (df["median_house_value"] > median_value).astype(int).values
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
class LogisticRegressionModel(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.linear = nn.Linear(n_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
epoch = 50
for ep in range(epoch):
    y_hat = model(X)
    loss = criterion(y_hat, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if ep % 10 == 0:
        print(f"Epoch {ep}, loss = {loss.item()}")
w_torch = model.linear.weight.detach().numpy().flatten()
b_torch = model.linear.bias.detach().numpy().item()

print("w =", w_torch)
print("b =", b_torch)


Epoch 0, loss = 0.7311917543411255
Epoch 10, loss = 0.7247227430343628
Epoch 20, loss = 0.7188087701797485
Epoch 30, loss = 0.7133939862251282
Epoch 40, loss = 0.7084267735481262
w = [-0.11717667 -0.12951815  0.22518814  0.2336286   0.15178728  0.1575226
 -0.01326949  0.07524105  0.00134657  0.09833094 -0.20280932  0.16923928]
b = 0.14955110847949982
